In [1]:
# from BCBio.GFF import GFFExaminer

https://gffpandas.readthedocs.io/en/latest/tutorial.html

In [340]:
import gffpandas.gffpandas as gffpd
import pandas as pd
# Import Biopython tools for running local BLASTX
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import SeqIO

In [2]:
annotation = gffpd.read_gff3('YPS1009_S288C_v5_polished.gff')
annot_s288c = gffpd.read_gff3('saccharomyces_cerevisiae_R64-3.gff')

C:\Users\user\anaconda3\lib\site-packages\gffpandas\gffpandas.py:32: DtypeWarning: Columns (1,2,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_table(self._gff_file, comment='#',


In [7]:
features = annotation.df['type'].value_counts().to_frame()
features = features.rename(columns = {'type': 'yps1009'})

In [10]:
features = features.join(annot_s288c.df['type'].value_counts().to_frame())
features = features.rename(columns={'type': 'S288C'})

In [11]:
features

,yps1009,S288C
CDS,7164,7066
gene,6691,6607
mRNA,6691,11119
ARS,554,543
noncoding_exon,512,489
long_terminal_repeat,361,384
tRNA,323,299
tRNA_gene,323,299
ARS_consensus_sequence,199,196
telomere,98,64


# Coding sequence

In [298]:
CDS_df = annotation.filter_feature_of_type(['CDS'])
CDS_attr = CDS_df.attributes_to_columns()

In [306]:
CDS_attr['Name short'] = CDS_attr['Name'].str.replace('_CDS', '')

In [310]:
len(set(gene['Name']) & set(CDS_attr['Name short']))

6565

In [327]:
CDS_attr.shape

(7164, 16)

In [330]:
gene.shape

(6691, 23)

In [331]:
gene['extra_copy_number'].value_counts()

0    6563
1      91
2      19
3      10
4       3
5       2
6       1
Name: extra_copy_number, dtype: int64

In [329]:
CDS_attr['extra_copy_number'].value_counts()

0    7020
1     107
2      20
3      11
4       3
5       2
6       1
Name: extra_copy_number, dtype: int64

In [312]:
len(set(CDS_attr['Name short']) - set(gene['Name']))

97

In [314]:
diff = set(CDS_attr['Name short']) - set(gene['Name'])

In [315]:
CDS_attr[CDS_attr['Name short'].isin(diff)].to_excel('CDS_not_gene')

,seq_id,source,type,start,end,score,strand,phase,attributes,ID,Name,Parent,extra_copy_number,orf_classification,protein_id,Name short
410,chrIII_tig00004936,Liftoff,CDS,140850,142166,.,-,.,ID=CDS_140_1;Parent=YBL100W-A_transposable_ele...,CDS_140_1,YBL100W-A_CDS,YBL100W-A_transposable_element_1,1,None,UniProtKB:Q12260_1,YBL100W-A
1128,chrII_tig00000010,Liftoff,CDS,185379,186695,.,+,.,ID=CDS_1322;Parent=YDR210W-A_transposable_elem...,CDS_1322,YDR210W-A_CDS,YDR210W-A_transposable_element,0,None,UniProtKB:Q03483,YDR210W-A
1131,chrII_tig00000010,Liftoff,CDS,185379,186695,.,+,.,ID=CDS_1380;Parent=YDR261W-A_transposable_elem...,CDS_1380,YDR261W-A_CDS,YDR261W-A_transposable_element,0,None,UniProtKB:Q99303,YDR261W-A
1134,chrII_tig00000010,Liftoff,CDS,185379,186695,.,+,.,ID=CDS_2668;Parent=YGR161W-A_transposable_elem...,CDS_2668,YGR161W-A_CDS,YGR161W-A_transposable_element,0,None,UniProtKB:P0CX62,YGR161W-A
3667,chrIV_tig00000001,Liftoff,CDS,700770,701156,.,-,.,ID=CDS_1238;Parent=YDR134C;Name=YDR134C_CDS;pr...,CDS_1238,YDR134C_CDS,YDR134C,0,None,UniProtKB:Q7LHD1,YDR134C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19307,chrXVI-2_tig00000040,Liftoff,CDS,58988,60304,.,-,.,ID=CDS_668;Parent=YCL020W_transposable_element...,CDS_668,YCL020W_CDS,YCL020W_transposable_element,0,None,UniProtKB:P25383,YCL020W
21015,chrXV_tig00000007,Liftoff,CDS,37854,39599,.,-,.,ID=CDS_5850;Parent=YOL153C;Name=YOL153C_CDS;pr...,CDS_5850,YOL153C_CDS,YOL153C,0,None,UniProtKB:P0C155,YOL153C
21165,chrXV_tig00000007,Liftoff,CDS,112411,113727,.,+,.,ID=CDS_140;Parent=YBL100W-A_transposable_eleme...,CDS_140,YBL100W-A_CDS,YBL100W-A_transposable_element,0,None,UniProtKB:Q12260,YBL100W-A
21698,chrXV_tig00000007,Liftoff,CDS,396557,396766,.,+,.,ID=CDS_6054;Parent=YOR031W;Name=YOR031W_CDS;pr...,CDS_6054,YOR031W_CDS,YOR031W,0,None,UniProtKB:P41902,YOR031W


# Centromere

In [105]:
CEN_df = annotation.filter_feature_of_type(['centromere'])
CEN_attr = CEN_df.attributes_to_columns()

In [108]:
CEN_attr = CEN_attr[CEN_attr['Alias'].notna()]
CEN_attr = CEN_attr.sort_values(by=['Alias'])

In [109]:
CEN_attr

,seq_id,source,type,start,end,score,strand,phase,attributes,Alias,...,Note,Parent,copy_num_ID,coverage,curie,dbxref,display,extra_copy_number,gene,sequence_ID
6319,chrI_tig00000033,Liftoff,centromere,148004,148121,.,+,.,ID=CEN1;Name=CEN1;gene=CEN1;Alias=CEN1;Note=Ch...,CEN1,...,Chromosome%20I%20centromere,None,CEN1_0,1.0,SGD:S000006463,SGD:S000006463,Chromosome%20I%20centromere,0,CEN1,0.957
23281,chrX_tig00001098,Liftoff,centromere,432174,432290,.,-,.,ID=CEN10;Name=CEN10;gene=CEN10;Alias=CEN10;Not...,CEN10,...,Chromosome%20X%20centromere,None,CEN10_0,0.983,SGD:S000006471,SGD:S000006471,Chromosome%20X%20centromere,0,CEN10,0.957
18292,chrXI_tig00004932,Liftoff,centromere,440039,440156,.,-,.,ID=CEN11;Name=CEN11;gene=CEN11;Alias=CEN11;Not...,CEN11,...,Chromosome%20XI%20centromere,None,CEN11_0,1.0,SGD:S000006472,SGD:S000006472,Chromosome%20XI%20centromere,0,CEN11,0.983
12444,chrXII-2_tig00004934,Liftoff,centromere,144863,144981,.,-,.,ID=CEN12;Name=CEN12;gene=CEN12;Alias=CEN12;Not...,CEN12,...,Chromosome%20XII%20centromere,None,CEN12_0,0.991,SGD:S000006473,SGD:S000006473,Chromosome%20XII%20centromere,0,CEN12,0.958
13618,chrXIII-2_tig00000029,Liftoff,centromere,251334,251453,.,+,.,ID=CEN13;Name=CEN13;gene=CEN13;Alias=CEN13;Not...,CEN13,...,Chromosome%20XIII%20centromere,None,CEN13_0,1.0,SGD:S000006474,SGD:S000006474,Chromosome%20XIII%20centromere,0,CEN13,0.966
17197,chrXIV_tig00004929,Liftoff,centromere,576570,576687,.,+,.,ID=CEN14;Name=CEN14;gene=CEN14;Alias=CEN14;Not...,CEN14,...,Chromosome%20XIV%20centromere,None,CEN14_0,1.0,SGD:S000006475,SGD:S000006475,Chromosome%20XIV%20centromere,0,CEN14,0.966
21577,chrXV_tig00000007,Liftoff,centromere,333875,333993,.,-,.,ID=CEN15;Name=CEN15;gene=CEN15;Alias=CEN15;Not...,CEN15,...,Chromosome%20XV%20centromere,None,CEN15_0,0.991,SGD:S000006476,SGD:S000006476,Chromosome%20XV%20centromere,0,CEN15,0.941
20176,chrXVI_tig00000013,Liftoff,centromere,351056,351172,.,+,.,ID=CEN16;Name=CEN16;gene=CEN16;Alias=CEN16;Not...,CEN16,...,Chromosome%20XVI%20centromere,None,CEN16_0,1.0,SGD:S000006477,SGD:S000006477,Chromosome%20XVI%20centromere,0,CEN16,0.974
1200,chrII_tig00000010,Liftoff,centromere,226275,226391,.,+,.,ID=CEN2;Name=CEN2;gene=CEN2;Alias=CEN2;Note=Ch...,CEN2,...,Chromosome%20II%20centromere,None,CEN2_0,1.0,SGD:S000006464,SGD:S000006464,Chromosome%20II%20centromere,0,CEN2,0.965
339,chrIII_tig00004936,Liftoff,centromere,108185,108301,.,+,.,ID=CEN3;Name=CEN3;gene=CEN3;Alias=CEN3;Note=Ch...,CEN3,...,Chromosome%20III%20centromere,None,CEN3_0,1.0,SGD:S000006465,SGD:S000006465,Chromosome%20III%20centromere,0,CEN3,0.957


# Telomere

In [110]:
telomere_df = annotation.filter_feature_of_type(['telomere'])
telomere_attr = telomere_df.attributes_to_columns()

In [114]:
telomere_attr = telomere_attr[telomere_attr['Parent'].notna()]
telomere_attr = telomere_attr.sort_values(by=['ID', 'start'])

## Make df by features YPS1009

In [49]:
TE_df = annotation.filter_feature_of_type(['transposable_element_gene'])
sno_df = annotation.filter_feature_of_type(['snoRNA_gene'])
tRNA_df = annotation.filter_feature_of_type(['tRNA_gene'])
gene_df = annotation.filter_feature_of_type(['gene'])
LTR_retroP_df = annotation.filter_feature_of_type(['LTR_retrotransposon'])

In [14]:
gene_attr = gene_df.attributes_to_columns()
gene = gene_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs']]

In [15]:
gene_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term',
       'copy_num_ID', 'coverage', 'curie', 'dbxref', 'display',
       'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs'],
      dtype='object')

In [16]:
gene_attr['valid_ORFs'].value_counts()
# so valid_ORFs in gene does count itself and I should look carefully at the 439 genes with no valid ORF

1    6165
0     524
Name: valid_ORFs, dtype: int64

In [301]:
# number of annotated genes without potential duplicates
# Doesn't works, we want duplicates
len(set(gene['Name']))

6565

In [332]:
gene.shape

(6691, 23)

In [31]:
TE_attr = TE_df.attributes_to_columns()
sno_attr = sno_df.attributes_to_columns()
tRNA_attr = tRNA_df.attributes_to_columns()
LTR_retroP_attr = LTR_retroP_df.attributes_to_columns()

In [32]:
LTR_retroP_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'copy_num_ID', 'coverage',
       'curie', 'dbxref', 'display', 'extra_copy_number', 'low_identity',
       'sequence_ID'],
      dtype='object')

In [34]:
TE = TE_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'sequence_ID', 'valid_ORFs']]

sno = sno_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

tRNA = tRNA_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'gene', 'sequence_ID']]

LTR_retroP = LTR_retroP_attr[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'score', 'strand', 'phase',
       'copy_num_ID', 'coverage', 'curie', 'display', 'extra_copy_number', 'sequence_ID']]

In [36]:
# This code just convert the chromosome numbering to integer and remove mitochondrial genes.
def lat_to_arabic(DF):
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch', 'MANY': 7}
    seq_id = DF['seq_id']
    ch = []
    contig = []
    for i in seq_id:
        chrom, cont = i.split('_')
        contig.append(cont)
        chrom = chrom.replace('chr', '')
        if '-' in chrom:
            chrom, e = chrom.split('-')
        ch.append(Latin_to_arabic[chrom])
    DF['Chromosome'] = ch
    DF['Contig'] = contig
    return DF

In [37]:
gene = lat_to_arabic(gene)
TE = lat_to_arabic(TE)
sno = lat_to_arabic(sno)
tRNA = lat_to_arabic(tRNA)
LTR_retroP = lat_to_arabic(LTR_retroP)

C:\Users\user\AppData\Local\Temp\ipykernel_2116\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_2116\3764198105.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Contig'] = contig
C:\Users\user\AppData\Local\Temp\ipykernel_2116\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [38]:
def clean20(DF):
    note = DF['Note']
    dis = DF['display']
    clean = []
    display = []
    for n in note:
        clean.append(n.replace('%20', ' '))
    for d in dis:
        display.append(str(d).replace('%20', ' '))
    DF['Note'] = clean
    DF['display'] = display
    return DF

In [39]:
gene = clean20(gene)
TE = clean20(TE)
sno = clean20(sno)
tRNA = clean20(tRNA)
LTR_retroP = clean20(LTR_retroP)

C:\Users\user\AppData\Local\Temp\ipykernel_2116\741321415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Note'] = clean
C:\Users\user\AppData\Local\Temp\ipykernel_2116\741321415.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['display'] = display


## Df by attribute for S288C

In [40]:
TE_df_s288c = annot_s288c.filter_feature_of_type(['transposable_element_gene'])
sno_df_s288c = annot_s288c.filter_feature_of_type(['snoRNA_gene'])
tRNA_df_s288c = annot_s288c.filter_feature_of_type(['tRNA_gene'])
gene_df_s288c = annot_s288c.filter_feature_of_type(['gene'])
LTR_retroP_df_s288c = annot_s288c.filter_feature_of_type(['LTR_retrotransposon'])

In [41]:
gene_attr_s288c = gene_df_s288c.attributes_to_columns()
TE_attr_s288c = TE_df_s288c.attributes_to_columns()
sno_attr_s288c = sno_df_s288c.attributes_to_columns()
tRNA_attr_s288c = tRNA_df_s288c.attributes_to_columns()
LTR_retroP_attr_s288c = LTR_retroP_df_s288c.attributes_to_columns()

In [42]:
LTR_retroP_attr_s288c.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'curie', 'dbxref',
       'display'],
      dtype='object')

In [43]:
gene_s288c = gene_attr_s288c[['seq_id', 'start', 'end', 'Name', 'gene',  'Alias',  'Note', 'Ontology_term', 'strand', 'phase', 'curie',
        'orf_classification', 'display']]

TE_s288c = TE_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

sno_s288c = sno_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

tRNA_s288c = tRNA_attr_s288c[['seq_id', 'start', 'end', 'Name',  'Alias',  'Note', 'Ontology_term', 'score', 'strand', 'phase',
      'curie', 'display']]

LTR_retroP_s288c = LTR_retroP_attr_s288c[['seq_id', 'start', 'end',  'ID', 'Name',  'Alias', 'score', 'strand', 'phase',
       'attributes', 'Note', 'curie', 'display']]

In [44]:
def lat_to_arabic_S288(df):
    # This code just convert the chromosome numbering to integer and remove mitochondrial genes.
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch'}
    chLat = df['seq_id']
    ch = []
    for chrom in chLat:
        chrom = chrom.replace('chr', '')
        ch.append(Latin_to_arabic[chrom])
    df['Chromosome'] = ch
    return df

In [45]:
gene_s288c = lat_to_arabic_S288(gene_s288c)
TE_s288c = lat_to_arabic_S288(TE_s288c)
sno_s288c = lat_to_arabic_S288(sno_s288c)
tRNA_s288c = lat_to_arabic_S288(tRNA_s288c)
LTR_retroP_s288c = lat_to_arabic_S288(LTR_retroP_s288c)

C:\Users\user\AppData\Local\Temp\ipykernel_2116\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_2116\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_2116\760545625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [46]:
gene_s288c = clean20(gene_s288c)
TE_s288c = clean20(TE_s288c)
sno_s288c = clean20(sno_s288c)
tRNA_s288c = clean20(tRNA_s288c)
LTR_retroP_s288c = clean20(LTR_retroP_s288c)

C:\Users\user\AppData\Local\Temp\ipykernel_2116\741321415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Note'] = clean
C:\Users\user\AppData\Local\Temp\ipykernel_2116\741321415.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['display'] = display


# TE 

### IMPORTANT:

There are several TE mapped to the exact same location. Probably because map to 2 genes that have a very similar sequence!
That's bad, I need to remove them. DO MANUALLY. --> Only 17 unique TE found!!!!

In [117]:
TE.to_excel('TE_liftoff_v5_yps1009.xlsx')

## LTR Retrotransposons

In [65]:
LTR_RP_count = LTR_retroP['Chromosome'].value_counts().to_frame()
LTR_RP_count = LTR_RP_count.rename(columns = {'Chromosome' : 'yps1009 LTR retrotransposon'})
LTR_RP_count_s288 = LTR_retroP_s288c['Chromosome'].value_counts().to_frame()
LTR_RP_count_s288 = LTR_RP_count_s288.rename(columns= {'Chromosome': 'S288C LTR retroposon'})
LTR_RP_count = LTR_RP_count_s288.join(LTR_RP_count)

# IMPORTANT 
Overlapping features, to filter manually 

In [76]:
LTR_retroP.sort_values(by=['Chromosome', 'start'])

,seq_id,start,end,Name,Alias,Note,score,strand,phase,copy_num_ID,coverage,curie,display,extra_copy_number,sequence_ID,Chromosome,Contig
6374,chrI_tig00000033,177542,183707,YDRWTy2-3,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,+,.,YDRWTy2-3_0,0.862,SGD:S000006885,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.835,1,tig00000033
6375,chrI_tig00000033,177544,183707,YGRWTy2-2,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,+,.,YGRWTy2-2_0,0.861,SGD:S000006983,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.834,1,tig00000033
6376,chrI_tig00000033,177545,183707,YDRWTy2-2,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,+,.,YDRWTy2-2_0,0.860,SGD:S000006884,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.833,1,tig00000033
37,chrIII-2_tig00004935,423,4934,YCLWTy5-1,Ty5,Ty5 element%2C LTR retrotransposon of the Copi...,.,+,.,YCLWTy5-1_1,0.993,SGD:S000006831,Ty5 element%2C LTR retrotransposon of the Copi...,1,0.948,3,tig00004935
57,chrIII_tig00004936,790,4554,YCLWTy5-1,Ty5,Ty5 element%2C LTR retrotransposon of the Copi...,.,+,.,YCLWTy5-1_0,0.997,SGD:S000006831,Ty5 element%2C LTR retrotransposon of the Copi...,0,0.954,3,tig00004936
407,chrIII_tig00004936,136337,142463,YBLWTy2-1,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,-,.,YBLWTy2-1_0,0.856,SGD:S000006809,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.823,3,tig00004936
4750,chrIV_tig00000001,1297667,1303627,YLRWTy2-1,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,-,.,YLRWTy2-1_0,0.833,SGD:S000007100,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.812,4,tig00000001
4751,chrIV_tig00000001,1297674,1309617,YNLWTy1-2,Ty1,Ty1 element%2C LTR retrotransposon of the Copi...,.,-,.,YNLWTy1-2_0,0.830,SGD:S000007142,Ty1 element%2C LTR retrotransposon of the Copi...,0,0.426,4,tig00000001
4752,chrIV_tig00000001,1297677,1303627,YDRCTy2-1,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,-,.,YDRCTy2-1_0,0.832,SGD:S000006862,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.812,4,tig00000001
4753,chrIV_tig00000001,1297677,1304036,YFLWTy2-1,Ty2,Ty2 element%2C LTR retrotransposon of the Copi...,.,-,.,YFLWTy2-1_0,0.866,SGD:S000006930,Ty2 element%2C LTR retrotransposon of the Copi...,0,0.826,4,tig00000001


In [118]:
LTR_retroP.to_excel('Liftoff_LTR_retrop.xlsx')

How do I reconciliate these results? it somehow feels not right
compare to the TE Ultimate results. See TE_annotation jupyter notebook 

In [258]:
filteredRetroP = pd.read_excel('Liftoff_LTR_retrop_filtered.xlsx', index_col=0)

In [268]:
RetroP_count = filteredRetroP['Chromosome'].value_counts().to_frame()
RetroP_count = RetroP_count.rename(columns = {'Chromosome': 'yps1009 RetroP'})
RetroP_count_s = LTR_retroP_s288c['Chromosome'].value_counts().to_frame()
RetroP_count_s = RetroP_count_s.rename(columns = {'Chromosome': 'S288C RetroP'})

In [271]:
RetroP_count = pd.concat((RetroP_count_s, RetroP_count), axis=1)

In [272]:
RetroP_count.sort_index()

,S288C RetroP,yps1009 RetroP
1,1.0,1.0
2,3.0,NaN
3,2.0,3.0
4,8.0,2.0
5,2.0,1.0
6,1.0,NaN
7,6.0,2.0
8,2.0,1.0
9,1.0,NaN
10,3.0,NaN


In [273]:
RetroP_count.sum()

S288C RetroP      50.0
yps1009 RetroP    16.0
dtype: float64

# Compared to the filtered ReasonaTE

In [274]:
Reasona = pd.read_csv('ReasonaTE_filteredblast.txt', sep = '\t')

In [279]:
Reasona_count = Reasona['chromosome'].value_counts().to_frame()

In [282]:
Reasona_count.rename(columns={'chromosome': 'ReasonaTE yps RetroP'}, inplace=True)

In [283]:
RetroP_count = RetroP_count.join(Reasona_count)

In [284]:
RetroP_count

,S288C RetroP,yps1009 RetroP,ReasonaTE yps RetroP
4,8.0,2.0,4.0
7,6.0,2.0,2.0
12,5.0,1.0,1.0
16,5.0,1.0,1.0
13,4.0,NaN,NaN
15,4.0,2.0,2.0
2,3.0,NaN,1.0
10,3.0,NaN,1.0
14,3.0,NaN,NaN
3,2.0,3.0,1.0


In [285]:
RetroP_count.sum()

S288C RetroP            50.0
yps1009 RetroP          16.0
ReasonaTE yps RetroP    20.0
dtype: float64

In [286]:
combiTE = pd.read_csv('TE_liftoffv5_reasonaTE_combined.txt', sep='\t')

In [288]:
combi_count = combiTE['chromosome'].value_counts().to_frame()

In [289]:
combi_count.rename(columns={'chromosome': 'Reasona+Liftoff'}, inplace=True)

In [290]:
RetroP_count = RetroP_count.join(combi_count)

In [293]:
RetroP_count.sort_index()

,S288C RetroP,yps1009 RetroP,ReasonaTE yps RetroP,Reasona+Liftoff
1,1.0,1.0,2.0,2.0
2,3.0,NaN,1.0,1.0
3,2.0,3.0,1.0,3.0
4,8.0,2.0,4.0,4.0
5,2.0,1.0,2.0,2.0
6,1.0,NaN,2.0,2.0
7,6.0,2.0,2.0,2.0
8,2.0,1.0,1.0,1.0
9,1.0,NaN,NaN,NaN
10,3.0,NaN,1.0,1.0


# snoRNA

In [78]:
sno_count = sno['Chromosome'].value_counts().to_frame()
sno_count = sno_count.rename(columns = {'Chromosome': 'yps1009 snoRNA'})
sno_count_s = sno_s288c['Chromosome'].value_counts().to_frame()
sno_count_s = sno_count_s.rename(columns = {'Chromosome': 'S288C snoRNA'})
sno_count = sno_count_s.join(sno_count)

In [79]:
sno_count.to_excel('snoRNA_count_v5.xlsx')

In [80]:
sno_count

,S288C snoRNA,yps1009 snoRNA
13,13,13
15,11,11
12,7,7
7,6,6
16,6,6
5,5,5
10,5,5
11,5,5
14,5,5
3,4,4


# tRNA

In [81]:
tRNA['extra_copy_number'].value_counts()

0    299
1     24
Name: extra_copy_number, dtype: int64

In [316]:
tRNA_count = tRNA['Chromosome'].value_counts().to_frame()
tRNA_count = tRNA_count.rename(columns = {'Chromosome': 'yps1009 tRNA'})
tRNA_count_s = tRNA_s288c['Chromosome'].value_counts().to_frame()
tRNA_count_s = tRNA_count_s.rename(columns = {'Chromosome': 'S288C tRNA'})
tRNA_count = tRNA_count_s.join(tRNA_count)

In [83]:
tRNA_count.to_excel('tRNA_count_v5.xlsx')

In [319]:
tRNA.shape

(323, 19)

In [321]:
tRNA_s288c.shape

(299, 13)

# GENE

In [86]:
gene['orf_classification'].value_counts()

Verified           5250
Uncharacterized     731
Dubious             708
Name: orf_classification, dtype: int64

In [87]:
dubious = gene[gene['orf_classification'] == 'Dubious']
dubious['valid_ORFs'].value_counts()

1    518
0    190
Name: valid_ORFs, dtype: int64

In [88]:
uncharacterized = gene[gene['orf_classification'] == 'Uncharacterized']
uncharacterized['valid_ORFs'].value_counts()

1    559
0    172
Name: valid_ORFs, dtype: int64

In [333]:
gene_valid = gene[gene['valid_ORFs'] == '1']
gene_unvalid = gene[gene['valid_ORFs'] == '0']

gene_valid_count = gene_valid['Chromosome'].value_counts().to_frame()
gene_valid_count = gene_valid_count.rename(columns={'Chromosome': 'Number valid genes'})
gene_unvalid_count = gene_unvalid['Chromosome'].value_counts().to_frame()
gene_unvalid_count = gene_unvalid_count.rename(columns={'Chromosome': 'Number unvalid genes'})

In [334]:
unval_dic = {}
for ind, row in gene_unvalid.iterrows():
    if row['Chromosome'] in unval_dic:
        unval_dic[row['Chromosome']] = unval_dic[row['Chromosome']] + [row['Name']]
    else:
        unval_dic[row['Chromosome']] = [row['Name']]
        
gene_sum = gene_valid_count.join(gene_unvalid_count)
tmp = []
for ind in gene_sum.index:
    tmp.append(unval_dic[ind])
gene_sum['unvalid genes'] = tmp

In [335]:
missed = []
moved = []
count_miss = []
count_move = []
for Ch in gene_sum.index:
    s288 = gene_s288c[gene_s288c['Chromosome'] == Ch]
    yps = gene[gene['Chromosome'] == Ch]
    tmp = list(set(s288['Name']) - set(yps['Name']))
    miss = []
    mov = []
    others = gene[gene['Chromosome'] != Ch]
    for g in tmp:
        search = others[others['Name'] == g]
        if search.shape[0] >= 1:
            mov.append((g, list(search['Chromosome']), list(search['valid_ORFs'])))
        else:
            miss.append(g)  
    moved.append(mov)
    missed.append(miss)
    count_miss.append(len(miss))
    count_move.append(len(mov))

In [336]:
gene_sum['Number of missing genes'] = count_miss
gene_sum['Missing genes'] = missed
gene_sum['Number of moved genes'] = count_move
gene_sum['Moved genes, new chromosome(s), orf validity'] = moved

In [349]:
gene_sum.sum()

Number valid genes                                                                           6165
Number unvalid genes                                                                          524
unvalid genes                                   [YFL063W, YHL048C-A, YDL247W, YJR157W, YNL331C...
Number of missing genes                                                                        42
Missing genes                                   [YDR034C-A, YOR255W, YOR248W, YOR246C, YOR247W...
Number of moved genes                                                                         142
Moved genes, new chromosome(s), orf validity    [(YDR544C, [11], [0]), (YDR545W, [5], [1]), (Y...
dtype: object

In [347]:
flat_list = [item for sublist in missed for item in sublist]
missing_genes = gene_s288c[gene_s288c['Name'].isin(flat_list)]
missing_genes.to_excel('missing_genes_v5.xlsx')

In [343]:
# Blast
ids = []
seq = []
for seq_record in SeqIO.parse("fasta_file/orf_coding_all_R64-3-1_20210421.fasta", "fasta"):
    ids.append(str(seq_record.id))
    seq.append(str(seq_record.seq))

In [345]:
for g in flat_list:
    name = g + '.txt'
    with open(name , "w") as file:
        file.write('> ' + g + '\n')
        file.write(seq[ids.index(g)])
    outfile = 'Blast_results/' + g + '.tab'
    cmd_blastn = NcbiblastnCommandline(query= name, out= outfile, evalue=0.001, db= 'YPS1009_DB/YPS1009_DB')
    stdout, stderr = cmd_blastn()
    #print("STDOUT: %s" % stdout)
    #print("STDERR: %s" % stderr)

In [346]:
len(flat_list)

42

# Verified ORF

In [89]:
verified = gene[gene['orf_classification'] == 'Verified']

In [90]:
verified['valid_ORFs'].value_counts()

1    5088
0     162
Name: valid_ORFs, dtype: int64

In [91]:
verified_s288c = gene_s288c[gene_s288c['orf_classification'] == 'Verified']

In [92]:
verified_s288c.shape

(5216, 14)

In [93]:
verified_valid = verified[verified['valid_ORFs'] == '1']
verified_unvalid = verified[verified['valid_ORFs'] == '0']

verified_valid_count = verified_valid['Chromosome'].value_counts().to_frame()
verified_valid_count = verified_valid_count.rename(columns={'Chromosome': 'Number valid genes'})
verified_unvalid_count = verified_unvalid['Chromosome'].value_counts().to_frame()
verified_unvalid_count = verified_unvalid_count.rename(columns={'Chromosome': 'Number unvalid genes'})

In [94]:
unval_dic = {}
for ind, row in verified_unvalid.iterrows():
    if row['Chromosome'] in unval_dic:
        unval_dic[row['Chromosome']] = unval_dic[row['Chromosome']] + [row['Name']]
    else:
        unval_dic[row['Chromosome']] = [row['Name']]    

In [95]:
verified_sum = verified_valid_count.join(verified_unvalid_count)

In [96]:
tmp = []
for ind in verified_sum.index:
    tmp.append(unval_dic[ind])

In [97]:
verified_sum['unvalid genes'] = tmp

In [99]:
missed = []
moved = []
count_miss = []
count_move = []
for Ch in verified_sum.index:
    s288 = verified_s288c[verified_s288c['Chromosome'] == Ch]
    yps = verified[verified['Chromosome'] == Ch]
    tmp = list(set(s288['Name']) - set(yps['Name']))
    miss = []
    mov = []
    others = verified[verified['Chromosome'] != Ch]
    for g in tmp:
        search = others[others['Name'] == g]
        if search.shape[0] >= 1:
            mov.append((g, list(search['Chromosome']), list(search['valid_ORFs'])))
        else:
            miss.append(g)  
    moved.append(mov)
    missed.append(miss)
    count_miss.append(len(miss))
    count_move.append(len(mov))

In [100]:
verified_sum['Number of missing genes'] = count_miss
verified_sum['Missing genes'] = missed
verified_sum['Number of moved genes'] = count_move
verified_sum['Moved genes, new chromosome(s), orf validity'] = moved

In [101]:
verified_sum.to_excel('Liftoff_v5_verified_genes_summary.xlsx')

# Verified missing genes dataset

In [102]:
flat_list = [item for sublist in missed for item in sublist]

In [103]:
missing_genes = verified_s288c[verified_s288c['Name'].isin(flat_list)]

In [104]:
missing_genes.to_excel('missing_verified_genes_v5.xlsx')

# Duplicated genes



In [79]:
# Many duplicated genes in this version
gene['extra_copy_number'].value_counts()

0    6563
1      94
2      22
3      12
4       7
5       5
6       2
Name: extra_copy_number, dtype: int64

In [80]:
gene[gene['extra_copy_number'] > '1']

,seq_id,start,end,Name,Alias,Note,Ontology_term,score,strand,phase,...,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs,Chromosome,Contig
9,chrII-2_tig00004927,14242,14700,YGR293C,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious open reading frame,2,None,None,Dubious,None,0.989,0,2,tig00004927
21,chrII-2_tig00004927,17994,18290,YER188C-A,None,Putative protein of unknown function,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Putative protein of unknown function,3,None,None,Uncharacterized,None,0.943,1,2,tig00004927
24,chrIII-2_tig00004935,767,1021,YNR077C,None,Protein of unknown function%2C abundance chang...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Protein of unknown function%2C abundance chang...,3,None,None,Uncharacterized,None,0.918,1,3,tig00004935
27,chrIII-2_tig00004935,1323,2063,YCL076W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Dubious open reading frame,2,None,None,Dubious,None,0.981,0,3,tig00004935
30,chrIII-2_tig00004935,6426,8271,YCL073C,"GEX1,glutathione%20exchanger",Proton:glutathione antiporter%3B localized to ...,"GO:0005774,GO:0005886,GO:0015299,GO:0034775,GO...",.,-,.,...,Proton:glutathione antiporter,2,GEX1,None,Verified,None,0.990,0,3,tig00004935
33,chrIII_tig00004936,760,1014,YNR077C,None,Protein of unknown function%2C abundance chang...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Protein of unknown function%2C abundance chang...,2,None,None,Uncharacterized,None,0.918,1,3,tig00004936
36,chrIII_tig00004936,1316,2055,YCL076W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Dubious open reading frame,3,None,None,Dubious,None,0.979,0,3,tig00004936
651,chrIII_tig00004936,310848,311102,YNL337W,None,Dubious open reading frame%3B unlikely to enco...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Dubious open reading frame,4,None,None,Dubious,None,0.949,1,3,tig00004936
2097,chrII_tig00000010,789332,789467,YHL048C-A,None,Putative protein of unknown function%3B identi...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,+,.,...,Putative protein of unknown function,6,None,None,Uncharacterized,None,0.890,0,2,tig00000010
2106,chrIV_tig00000001,1219,1356,YHL048C-A,None,Putative protein of unknown function%3B identi...,"GO:0003674,GO:0005575,GO:0008150,SO:0000704",.,-,.,...,Putative protein of unknown function,4,None,None,Uncharacterized,None,0.920,0,4,tig00000001


In [82]:
dupl = gene[gene.duplicated(subset=['Name'], keep=False) == True]

In [83]:
dupl = dupl.sort_values(by = 'Name')

In [91]:
dupl.to_excel('duplicated_verified_genes.xlsx')

Duplicated genes:

ATP8/Q0080: Subunit 8 of the F0 sector of mitochondrial F1F0 ATP synthase; encoded on the mitochondrial genome; ATP8 and ATP6 mRNAs are not translated in the absence of the F1 sector of ATPase

OLI1/Q0130: F0-ATP synthase subunit c (ATPase-associated proteolipid); encoded on the mitochondrial genome; mutation confers oligomycin resistance; expression is specifically dependent on the nuclear genes AEP1 and AEP2 1 2 3 4
Name Description
OLIgomycin resistance 1

Both on mitochondria so not that relevant to my project.

PAU9/YBL108C-A: Protein of unknown function; member of the seripauperin multigene family encoded mainly in subtelomeric regions; SWAT-GFP and mCherry fusion proteins localize to the endoplasmic reticulum and vacuole respectively

YHR213W-A: Putative protein of unknown function; identified by gene-trapping, microarray-based expression analysis, and genome-wide homology searching 

# Analyse unvalid genes

6 partial mapping genes:
    YDL227C near telomere
    YMR106C middle of chromosome arm
    YNL322C near telomere and missing genes, suggesting GAP but could also be a fragment deletion.
    YOR244W middle of chromosome arm
    YOR245C middle of chromosome arm

### merge mRNA and gene data to get information on the type of issue in ORF

In [189]:
mRNA_df = annotation.filter_feature_of_type(['mRNA'])
mRNA_attr = mRNA_df.attributes_to_columns()
mRNA_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'ID', 'Name', 'Parent', 'conditions', 'dbxref',
       'extra_copy_number', 'inframe_stop_codon', 'matches_ref_protein',
       'missing_start_codon', 'missing_stop_codon', 'transcript_id',
       'valid_ORF'],
      dtype='object')

In [190]:
mRNA = mRNA_attr[['seq_id', 'start', 'end', 'ID', 'Name', 'Parent',
                  'conditions', 'extra_copy_number', 'inframe_stop_codon', 
                  'matches_ref_protein', 'missing_start_codon', 'missing_stop_codon', 
                  'transcript_id', 'valid_ORF']]

In [191]:
gene_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term',
       'copy_num_ID', 'coverage', 'curie', 'dbxref', 'display',
       'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs'],
      dtype='object')

In [192]:
gene2 = gene_attr[['seq_id', 'start', 'end',
       'Alias', 'ID', 'Name', 'copy_num_ID', 'coverage', 'display',
       'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs']]

In [193]:
gene2.head()

,seq_id,start,end,Alias,ID,Name,copy_num_ID,coverage,display,extra_copy_number,gene,low_identity,orf_classification,partial_mapping,sequence_ID,valid_ORFs
0,chrII-2_tig00004927,8297,10119,"MAL31,MAL3T,MALT,maltose%20permease",YBR298C_1,YBR298C,YBR298C_1,0.986,Maltose%20permease,1,MAL31,None,Verified,None,0.946,0
3,chrII-2_tig00004927,10675,10894,None,YBR298C-A_1,YBR298C-A,YBR298C-A_1,0.990,Putative%20protein%20of%20unknown%20function,1,None,None,Uncharacterized,None,0.981,0
6,chrII-2_tig00004927,10985,12717,"MAL32,MAL3S,MALS,alpha-glucosidase%20MAL32",YBR299W_1,YBR299W,YBR299W_1,0.986,Maltase%20%28alpha-D-glucosidase%29,1,MAL32,None,Verified,None,0.981,0
9,chrII-2_tig00004927,14242,14700,None,YGR293C_2,YGR293C,YGR293C_2,0.993,Dubious%20open%20reading%20frame,2,None,None,Dubious,None,0.989,0
12,chrII-2_tig00004927,14660,15022,"PAU12,seripauperin%20PAU12",YGR294W_1,YGR294W,YGR294W_1,1.0,Protein%20of%20unknown%20function,1,PAU12,None,Verified,None,0.983,1


In [194]:
mRNA.head()

,seq_id,start,end,ID,Name,Parent,conditions,extra_copy_number,inframe_stop_codon,matches_ref_protein,missing_start_codon,missing_stop_codon,transcript_id,valid_ORF
1,chrII-2_tig00004927,8297,10119,YBR298C_id001_1,YBR298C_id001,YBR298C_1,GAL,1,None,False,None,True,SGD:S000288108_1,False
4,chrII-2_tig00004927,10675,10894,YBR298C-A_mRNA_1,YBR298C-A_mRNA,YBR298C-A_1,None,1,None,False,None,True,RefSeq:NM_001184588.1_1,False
7,chrII-2_tig00004927,10985,12717,YBR299W_id001_1,YBR299W_id001,YBR299W_1,GAL,1,None,False,None,True,SGD:S000288109_1,False
10,chrII-2_tig00004927,14242,14700,YGR293C_mRNA_2,YGR293C_mRNA,YGR293C_2,None,2,True,False,None,None,None,False
13,chrII-2_tig00004927,14660,15022,YGR294W_id001_1,YGR294W_id001,YGR294W_1,GAL,1,None,True,None,None,SGD:S000289696_1,True


In [195]:
gene_mRNA = pd.merge(gene2, mRNA,  how='right', left_on='ID', right_on = 'Parent')

In [196]:
gene_mRNA['valid_ORF'].value_counts()

True     6167
False     524
Name: valid_ORF, dtype: int64

### Focus on Verified unvalid

In [197]:
gene_mRNA_verified = gene_mRNA[gene_mRNA['orf_classification']== 'Verified']
gene_mRNA_verified_unvalid = gene_mRNA_verified[gene_mRNA_verified['valid_ORF']== 'False']

In [208]:
gene_mRNA_verified_unvalid.drop_duplicates(subset=['Name_x'], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_2116\4212071888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_mRNA_verified_unvalid.drop_duplicates(subset=['Name_x'], inplace=True)


In [209]:
gene_mRNA_verified_unvalid.shape

(144, 30)

In [210]:
len(set(gene_mRNA_verified_unvalid['Name_x']))

144

In [211]:
gene_mRNA_verified_unvalid['missing_stop_codon'].value_counts()

True    101
Name: missing_stop_codon, dtype: int64

In [212]:
gene_mRNA_verified_unvalid['missing_start_codon'].value_counts()

True    17
Name: missing_start_codon, dtype: int64

In [213]:
gene_mRNA_verified_unvalid['inframe_stop_codon'].value_counts()

True    26
Name: inframe_stop_codon, dtype: int64

In [297]:
gene_mRNA_verified_unvalid[gene_mRNA_verified_unvalid['missing_start_codon'] == 'True']

,seq_id_x,start_x,end_x,Alias,ID_x,Name_x,copy_num_ID,coverage,display,extra_copy_number_x,...,Name_y,Parent,conditions,extra_copy_number_y,inframe_stop_codon,matches_ref_protein,missing_start_codon,missing_stop_codon,transcript_id,valid_ORF
157,chrIII_tig00004936,241438,245433,"SSK22,mitogen-activated%20protein%20kinase%20k...",YCR073C,YCR073C,YCR073C_0,1.0,MAP%20kinase%20kinase%20kinase%20of%20HOG1%20m...,0,...,YCR073C_mRNA,YCR073C,None,0,None,False,True,None,RefSeq:NM_001178783.1,False
676,chrIV_tig00000001,46438,46547,HO,YDL227C,YDL227C,YDL227C_0,0.062,Site-specific%20endonuclease,0,...,YDL227C_id001,YDL227C,YPD,0,None,False,True,None,SGD:S000293396,False
874,chrIV_tig00000001,381046,383253,"PRM7,YDL038C,pheromone-regulated%20protein%20PRM7",YDL039C,YDL039C,YDL039C_0,0.998,Pheromone-regulated%20protein,0,...,YDL039C_mRNA,YDL039C,None,0,None,False,True,None,RefSeq:NM_001180098.1,False
1610,chrIX_tig00000023,235689,236111,None,YIL060W,YIL060W,YIL060W_0,0.908,Mitochondrial%20protein%20of%20unknown%20function,0,...,YIL060W_mRNA,YIL060W,None,0,None,False,True,None,RefSeq:NM_001179410.3,False
1694,chrIX_tig00000023,373545,374108,MET28,YIR017C,YIR017C,YIR017C_0,1.0,bZIP%20transcriptional%20activator%20in%20the%...,0,...,YIR017C_id002,YIR017C,GAL,0,None,False,True,None,SGD:S000290068,False
1920,chrVII-2_tig00000038,38804,40006,"TAD1,tRNA-specific%20adenosine%20deaminase",YGL243W,YGL243W,YGL243W_0,1.0,tRNA-specific%20adenosine%20deaminase,0,...,YGL243W_id002,YGL243W,GAL,0,None,False,True,None,SGD:S000289433,False
2292,chrVIII_tig00000017,445786,446360,"CTR2,low-affinity%20Cu%20transporter",YHR175W,YHR175W,YHR175W_0,1.0,Low-affinity%20copper%20transporter%20of%20the...,0,...,YHR175W_id001,YHR175W,GAL,0,None,False,True,None,SGD:S000289883,False
2795,chrVI_tig00004938,12513,13592,"AAD3,putative%20aryl-alcohol%20dehydrogenase",YCR107W,YCR107W,YCR107W_0,0.982,Putative%20aryl-alcohol%20dehydrogenase,0,...,YCR107W_mRNA,YCR107W,None,0,None,False,True,None,RefSeq:NM_001178814.1,False
2887,chrVI_tig00004938,229268,229777,ULI1,YFR026C,YFR026C,YFR026C_0,1.0,Protein%20of%20unknown%20function,0,...,YFR026C_id001,YFR026C,GAL,0,None,False,True,None,SGD:S000289199,False
2937,chrV_tig00000015,680,5190,"YRF1-4,Y'%20element%20ATP-dependent%20helicase...",YLR466W,YLR466W,YLR466W_0,0.998,Helicase%20encoded%20by%20the%20Y'%20element%2...,0,...,YLR466W_mRNA,YLR466W,None,0,None,False,True,None,RefSeq:NM_001182354.3,False


# kick out duplicated genes who have a valid version

In [214]:
gene_mRNA_verified_valid = gene_mRNA_verified[gene_mRNA_verified['valid_ORF'] == 'True' ]

In [215]:
verified_valid_list = set(gene_mRNA_verified_valid['Name_x'])
verified_unvalid_list = set(gene_mRNA_verified_unvalid['Name_x'])
dupl_val_unval = verified_valid_list & verified_unvalid_list

In [216]:
# 16 genes have 1 duplicated unvalid version I can forget about
len(dupl_val_unval)

16

In [217]:
gene_mRNA_verified_unvalid = gene_mRNA_verified_unvalid[~gene_mRNA_verified_unvalid['Name_x'].isin(list(dupl_val_unval))]

In [350]:
gene_mRNA_verified_unvalid[gene_mRNA_verified_unvalid['missing_start_codon'] == 'True'].to_excel('missing_start.xlsx')

In [296]:
gene_mRNA_verified_unvalid[gene_mRNA_verified_unvalid['missing_stop_codon'] == 'True'].to_excel('missing_stop.xlsx')

In [220]:
gene_mRNA_verified_unvalid['missing_stop_codon'].value_counts()

True    88
Name: missing_stop_codon, dtype: int64

In [223]:
gene_mRNA_verified_unvalid['inframe_stop_codon'].value_counts()

True    23
Name: inframe_stop_codon, dtype: int64

# Inframe stop codon analysis

In [225]:
inframe_stop = gene_mRNA_verified_unvalid[gene_mRNA_verified_unvalid['inframe_stop_codon'] == 'True']

In [226]:
inframe_g = inframe_stop['Name_x']

In [228]:
inframe_288c = gene_s288c[gene_attr_s288c['Name'].isin(inframe_g)]

In [230]:
inframe_288c['lenght'] = inframe_288c['end'] - inframe_288c['start']

C:\Users\user\AppData\Local\Temp\ipykernel_2116\2401597198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inframe_288c['lenght'] = inframe_288c['end'] - inframe_288c['start']


In [234]:
inframe_stop['lenght'] = inframe_stop['end_x'] - inframe_stop['start_x']

C:\Users\user\AppData\Local\Temp\ipykernel_2116\2496349264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inframe_stop['lenght'] = inframe_stop['end_x'] - inframe_stop['start_x']


In [238]:
inframe_stop = inframe_stop[['Name_x','copy_num_ID', 'coverage', 'lenght']]
inframe_288c = inframe_288c[['Name', 'lenght']]

In [237]:
inframe_288c.columns

Index(['seq_id', 'start', 'end', 'Name', 'gene', 'Alias', 'Note',
       'Ontology_term', 'strand', 'phase', 'curie', 'orf_classification',
       'display', 'Chromosome', 'lenght'],
      dtype='object')

In [239]:
inframes_comp = pd.merge(inframe_288c, inframe_stop,  how='right', left_on='Name', right_on = 'Name_x')

In [241]:
inframes_comp['diff'] = inframes_comp['lenght_x'] - inframes_comp['lenght_y']

In [244]:
inframes_comp = inframes_comp.sort_values(by='diff')
inframes_comp = inframes_comp[['Name', 'lenght_x', 'lenght_y', 'diff']]
inframes_comp.rename(columns = {'lenght_x': 'length s288c gene', 'lenght_y': 'length yps gene'}, inplace = True)

In [246]:
inframes_comp.to_excel('inframes.xlsx')

## Unvalid ORFs to add to valid genes (because essential or valid ORF in y22 Liftoff)

In [356]:
gene_to_add = []
with open('missing_stop_start_genes_to_add.txt', 'r')as file:
    for l in file:
        gene_to_add.append(l.strip('\n'))

In [357]:
gene_to_add += list(inframe_stop['Name_x'])

In [359]:
to_add = verified_unvalid[verified_unvalid['Name'].isin(gene_to_add)]

In [361]:
verified_valid = verified_valid.append(to_add)

C:\Users\user\AppData\Local\Temp\ipykernel_2116\907560873.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  verified_valid = verified_valid.append(to_add)


In [362]:
verified_valid.shape

(5160, 23)

# Overlapped annotation, that's really problematic....
Find overlap and deal with them. keep the genes with highest % identity 

In [365]:
# find genes with major overlap (I only round the last digit of start and end position to zero, 
# so not perfect but will cover the biggest overlaps)
id_overlap = []
dupl = {}
for ind, row in verified_valid.iterrows():
    start = str(row['start'])
    start = start.replace(start[len(start) - 1:], '0')
    end = str(row['end'])
    end = end.replace(end[len(end) - 1:], '0')

    tmp = str(row['seq_id']) + start + end
    id_overlap.append(tmp)
    if tmp in dupl:
        dupl[tmp] += 1
    else:
        dupl[tmp] = 1
verified_valid['overlap id'] = id_overlap

In [366]:
filtered_dict = {k:v for (k,v) in dupl.items() if v > 1}
len(filtered_dict)

30

In [373]:
verified_valid['sequence_ID'] = verified_valid['sequence_ID'].astype(float)

In [378]:
ind = []
for k in filtered_dict.keys():
    tmp = verified_valid[verified_valid['overlap id'] == k]
    ind.append(tmp.nsmallest(1,"sequence_ID").index[0])

In [379]:
verified_valid.shape

(5160, 24)

In [380]:
verified_valid.drop(index=ind, inplace=True)
verified_valid.shape

(5130, 24)

In [381]:
# ok, I think I'm good now, let's export this and start the model

In [383]:
verified_valid_sh = verified_valid[['Name', 'gene', 'Chromosome']]

In [ ]:
verified_valid_sh.to_csv('yps1009_verified_genes_final.txt', sep = '\t', usin)